## Model Fitting

In [1]:
import warnings; warnings.filterwarnings("ignore");

import time
import pandas as pd
import numpy as np
import pickle as pickle
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import normalize

from scipy.sparse import hstack, coo_matrix, csr_matrix

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Load processed data

with open('XX_train.pkl', 'rb') as infile:
    X_train = pickle.load(infile)

with open('y_train.pkl', 'rb') as infile:
    y_train = pickle.load(infile)
    
with open('XX_test.pkl', 'rb') as infile:
    X_test = pickle.load(infile)
    
# with open('y_test.pkl', 'rb') as infile:
#     y_test = pickle.load(infile)    

### Feature Selection

In [3]:
X_train.shape

(74067, 650)

In [4]:
# We'll split our training data, 80% will go to "train" and 20% will go to "test":

# Split 80-20 train vs test data
train_x, test_x, train_y, test_y = train_test_split(X_train, 
                                                    y_train, 
                                                    test_size=0.20, 
                                                    random_state=0)

#train_x = normalize(train_x)
#test_x = normalize(test_x)
# train_y = train_y.values
# test_y = test_y.values
#train_y = train_y.astype('float64')
#test_y = test_y.astype('float64')

In [5]:
# Caculate the root mean square rooot
def rmse(predictions, targets):

    differences = predictions - targets                       #the DIFFERENCEs.

    differences_squared = differences ** 2                    #the SQUAREs of ^

    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^

    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^

    return rmse_val     #get the ^

In [6]:
# def nDCGScoresBased(pred, true, k):
# # pred: predicted rank *score* of the authors
# # - must be non-negative
# # - 0 means non-CEP
# # - the larger rank score, the better
# # true: ground truth log-score of the author
# # - pred and true should be aligned
# # k: number of ground truth CEPs
#     assert len(pred) == len(true)
#     assert len(true) >= k

#     # NOTE(hfang): ignore negative scores
#     true = np.array(true)
#     true[true < 0] = 0
#     assert sum(true > 0) >= k

#     # compute ideal DCG
#     true_sorted = sorted(true, reverse=True)
#     IDCGk = 0
#     for r, x in enumerate(true_sorted[:k]):
#         IDCGk += x / np.log2(r + 2)

#     # compute actual DCG
#     pred = np.array(pred)
#     m = min(sum(pred != 0), k)
#     assert sum(pred < 0) == 0
#     temp = pred.argsort()[::-1]
#     rank_pred = np.empty(len(pred), int)
#     rank_pred[temp] = np.arange(len(pred))
#     assert (pred[rank_pred < m] > 0).all()
#     predscore_with_rank = sorted(zip(rank_pred, true), \
#             key=lambda x:x[0])
#     DCGk = 0
#     for r, x in predscore_with_rank[:m]:
#         DCGk += x / np.log2(r + 2)
        
#     score = DCGk/IDCGk
#     return score

In [7]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
import re
import os
from scipy.stats import pearsonr

### NN 

In [6]:
from sklearn.neural_network import MLPRegressor
start_time = time.time()

nn_model = MLPRegressor(hidden_layer_sizes=(140, 70), 
                        activation='logistic', 
                        solver='lbfgs', 
                        alpha=0.5, #l2 penalty 
                        batch_size='auto', 
                        learning_rate='constant', 
                        learning_rate_init=0.001, 
                        power_t=0.5, 
                        max_iter=500, 
                        shuffle=True, 
                        random_state=2017, 
                        tol=0.0001, 
                        verbose=False, 
                        warm_start=True, 
                        momentum=0.9, 
                        nesterovs_momentum=True, 
                        early_stopping=False, 
                        validation_fraction=0.01, 
                        beta_1=0.9, 
                        beta_2=0.999, 
                        epsilon=1e-08).fit(train_x, train_y)

#pickle.dump(nn_model,open('Neural_Network_model.sav', 'wb'))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- time use: 1.81 minutes ---


In [7]:
start_time = time.time()
predict_nn = nn_model.predict(test_x)
print ("RMSE on test dataset = %.10f" % (rmse(predict_nn, test_y)))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

RMSE on test dataset = 0.5220361982
--- time use: 0.0 minutes ---


### Extra Tree 

In [8]:
from sklearn.ensemble import ExtraTreesRegressor
start_time = time.time()
extra_model = ExtraTreesRegressor(bootstrap=False, 
                                  max_depth=12, 
                                  min_samples_leaf=2,
                                  min_samples_split=6,
                                  n_estimators=250,
                                  n_jobs=-1,
                                  criterion='mse',
                                  min_weight_fraction_leaf=0.0,
                                  max_features='auto',
                                  warm_start=False,
                                  random_state=2017).fit(train_x, train_y)
#pickle.dump(extra_model,open('extra_tree_model.sav', 'wb'))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- time use: 2.15 minutes ---


In [9]:
start_time = time.time()

predict_xt = extra_model.predict(test_x)
print ("RMSE on test dataset = %.10f" % (rmse(predict_xt, test_y)))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

RMSE on test dataset = 0.4519042528
--- time use: 0.0 minutes ---


### GradientBoostingRegressor

In [10]:
from sklearn.ensemble import GradientBoostingRegressor

start_time = time.time()
gb_model = GradientBoostingRegressor(loss='ls', 
                                 learning_rate=0.05, 
                                 n_estimators=300, 
                                 subsample=1.0, 
                                 criterion='friedman_mse', 
                                 min_samples_split=2, 
                                 min_samples_leaf=15, 
                                 min_weight_fraction_leaf=0.0, 
                                 max_depth=6, 
                                 min_impurity_split=1e-07, 
                                 init=None, 
                                 random_state=2017, 
                                 max_features=None, 
                                 alpha=0.9, 
                                 verbose=0, 
                                 max_leaf_nodes=None, 
                                 warm_start=False, 
                                 presort='auto').fit(train_x, train_y)
#pickle.dump(gb_model,open('GBRegressor_model.sav', 'wb'))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- time use: 46.82 minutes ---


In [11]:
start_time = time.time()
predict_gb = gb_model.predict(test_x)
print ("RMSE on test dataset = %.10f" % (rmse(predict_gb, test_y)))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

RMSE on test dataset = 0.4434266726
--- time use: 0.01 minutes ---


### XGBoostRegressor

In [11]:
import xgboost as xgb

start_time = time.time()
#xgbclassifier
xgb_model1 = xgb.XGBRegressor(max_depth=9, 
                             learning_rate=0.075, 
                             n_estimators=100, 
                             silent=True, 
                             objective='reg:linear', 
                             nthread=-1, 
                             gamma=0, 
                             min_child_weight=10,
                             max_delta_step=0, 
                             subsample=1, 
                             colsample_bytree=1, 
                             colsample_bylevel=1, 
                             reg_alpha=0, 
                             reg_lambda=1, 
                             scale_pos_weight=1, 
                             base_score=0.5, 
                             seed=2017, 
                             missing=None).fit(train_x, train_y)
#pickle.dump(xgb_model,open('XGBRegressor_model.sav', 'wb'))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- time use: 0.97 minutes ---


In [12]:
start_time = time.time()

# predict
predict_xgb1 = xgb_model1.predict(test_x)
print ("RMSE on test dataset = %.10f" % (rmse(predict_xgb1, test_y)))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

RMSE on test dataset = 0.4424128379
--- time use: 0.0 minutes ---


### xgboost 

In [25]:
start_time = time.time()

dtrain = xgb.DMatrix(train_x, label=train_y)
#dval = xgb.DMatrix(X_val_FM, label=target_val)
dtest = xgb.DMatrix(test_x, test_y)

evallist  = [(dtrain,'train'), (dtrain,'train')]
bst_param = {'max_depth':10,
         'eta':0.1,
         'min_child_weight':1,
         'max_delta_step':0,
         'gamma':1,
         'lambda':1,
         'alpha':3,
         'colsample_bytree':0.3,
         'subsample':1,
         'eval_metric':'rmse',
         'maximize':False,
         'nthread':4}
nrounds = 500
xgb_fit = xgb.train(bst_param, 
                    dtrain, 
                    nrounds, 
                    evals = evallist, 
                    early_stopping_rounds = 20, 
                    verbose_eval = 50)

print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

[0]	train-rmse:1.77198	train-rmse:1.77198
Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.

Will train until train-rmse hasn't improved in 20 rounds.
[50]	train-rmse:0.376216	train-rmse:0.376216
[100]	train-rmse:0.344028	train-rmse:0.344028
[150]	train-rmse:0.329465	train-rmse:0.329465
[200]	train-rmse:0.32233	train-rmse:0.32233
[250]	train-rmse:0.31819	train-rmse:0.31819
[300]	train-rmse:0.315029	train-rmse:0.315029
[350]	train-rmse:0.313034	train-rmse:0.313034
[400]	train-rmse:0.310896	train-rmse:0.310896
[450]	train-rmse:0.309349	train-rmse:0.309349
--- time use: 4.62 minutes ---


In [26]:
start_time = time.time()
predict_bst= xgb_fit.predict(dtest)
print ("RMSE on test dataset = %.10f" % (rmse(predict_bst, test_y)))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

RMSE on test dataset = 0.4387737557
--- time use: 0.0 minutes ---


### Adaboost 

In [27]:
from sklearn.ensemble import AdaBoostRegressor
start_time = time.time()

ada_model = AdaBoostRegressor(base_estimator=None, 
                               n_estimators=90, 
                               learning_rate=0.5, 
                               random_state=2017).fit(train_x,train_y)
#pickle.dump(ada_model,open('AdaBoost_model.sav', 'wb'))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- time use: 4.24 minutes ---


In [28]:
start_time = time.time()

# pridiction
predict_ada = ada_model.predict(test_x)
print ("RMSE on test dataset = %.10f" % (rmse(predict_ada, test_y)))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

RMSE on test dataset = 0.4826711222
--- time use: 0.0 minutes ---


### Random Forest

In [61]:
from sklearn.ensemble import RandomForestRegressor

start_time = time.time()
# Initialize the Random Forest model
rf_model = RandomForestRegressor(n_estimators=200,
                                  max_features = 0.55,
                                  min_samples_leaf = 12,
                                  oob_score = True,
                                  max_depth = 15,
                                  n_jobs = -1,
                                  random_state = 2017).fit(train_x, train_y)
#pickle.dump(rf_model,open('random_forest_model.sav', 'wb'))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- time use: 4.01 minutes ---


In [62]:
start_time = time.time()
# predict
predict_rf = rf_model.predict(test_x)
print ("RMSE on test dataset = %.10f" % (rmse(predict_rf, test_y)))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

RMSE on test dataset = 0.4481836366
--- time use: 0.0 minutes ---


### Linear Regression (Baseline）

In [32]:
from sklearn.linear_model import LinearRegression
start_time = time.time()
lr_model = LinearRegression(n_jobs = -1).fit(train_x, train_y)
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- time use: 0.05 minutes ---


In [59]:
start_time = time.time()
predict_linear = lr_model.predict(test_x)
print ("RMSE on test dataset = %.2f" % (rmse(predict_linear, test_y)))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

RMSE on test dataset = 0.46
--- time use: 0.0 minutes ---


### Bagging Regressor 

In [53]:
from sklearn.ensemble import BaggingRegressor
start_time = time.time()

bag = BaggingRegressor(xgb_model1, 
                       n_estimators=1, 
                       random_state=2017,
                       n_jobs=-1,
                       verbose=True).fit(train_x,train_y)
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- time use: 1.13 minutes ---


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min finished


In [54]:
start_time = time.time()
# predict
predict_bag = bag.predict(test_x)
print ("RMSE on test dataset = %.10f" % (rmse(predict_bag, test_y)))
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))

RMSE on test dataset = 0.4521598698
--- time use: 0.0 minutes ---


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished


## Model Ensemble 

In [106]:
x_train_stack = np.stack((0.1*np.log1p(predict_bag), 
                                      0.2*np.log1p(predict_rf), 
                                      0.3*np.log1p(predict_xgb1),
                                      0.4*np.log1p(predict_bst)
                                      #np.log1p(predict_gb)
                                      ), axis = 1)
d_stack = xgb.DMatrix(x_train_stack)

In [113]:
x_train_stack.shape

(14814, 4)

In [119]:
# linear_stack = LinearRegression(n_jobs = -1).fit(x_train_stack, train_y)

In [104]:
from sklearn.linear_model import LinearRegression
evallist  = [(x_train_stack,'train'), (x_train_stack,'train')]
stack_param = {'max_depth':10,
         'eta':0.1,
         'min_child_weight':1,
         'max_delta_step':0,
         'gamma':1,
         'lambda':1,
         'alpha':3,
         'colsample_bytree':0.3,
         'subsample':1,
         'eval_metric':'rmse',
         'maximize':False,
         'nthread':4}
nrounds = 500
stack_model = xgb.train(stack_param, 
                          dtrain, 
                          nrounds, 
                          evals = evallist, 
                          early_stopping_rounds = 20, 
                          verbose_eval = 50)
print("--- time use: %s minutes ---" % round(((time.time() - start_time)/60),2))